## Section ID Data

Section ID data was not limited to top-level (H2) sections during data capture, requiring post-capture processing for section ID click data. Post-processing included scraping section data from HTML pages. There is a potential for data loss since scraping occured after click capture. Only WPM pages were scraped on 2019-04-23 PDT, meaning no comparison to W pages is possible. Extracting section data from dump files is another option, although recreating section IDs from text is problematic, making the approach here preferred. 

Example of capture issue: https://en.wikipedia.org/wiki/Hepatitis#Signs_and_symptoms.
Clicks on links under "Acute hepatitis" were captured with section_id Acute_hepatitis, not Signs_and_symptoms.

Post-capture data augmentation: H2 sections were extracted from public HTML of WPM pages 2019-04-23

See [section-extraction.ipynb](section-extraction.ipynb) for extraction details


In [1]:
# basic defaults, including study dates, common SQL exclusions and parquet files for anonymized data
%run -i 'data-defaults.py'

In [3]:
# count of top-level (H2) section IDs for WP:M pages only
pm_sections_query = """
SELECT section_h2, count(*) count
FROM
    ryanmax.wpm_sections 
GROUP BY section_h2
ORDER BY count desc, section_h2
"""

pm_sections = spark.sql(pm_sections_query)
pm_sections_rdd = pm_sections.rdd
pm_sections_df = sqlContext.createDataFrame(pm_sections_rdd)
pm_sections_df.toPandas()


,section_h2,count
0,References,31533
1,External_links,18683
2,See_also,13482
3,Diagnosis,8625
4,History,8120
5,Treatment,7245
6,Signs_and_symptoms,4657
7,Causes,4413
8,Further_reading,3284
9,Society_and_culture,3055


In [4]:
# Total count of events (by all event types) for each top-level (H2) section ID for WP:M pages only
pm_section_events_query = """
SELECT wpm_sections.section_h2, action, count(*) count
FROM 
    citationusage
    LEFT JOIN ryanmax.wpm_sections 
        ON 
        wpm_sections.page_id = citationusage.page_id 
        AND wpm_sections.section_id = citationusage.section_id
WHERE
    wiki = 'enwiki'
    AND citationusage.page_id IN (
                            SELECT DISTINCT page_id 
                            FROM ryanmax.projmed_with_extlinks 
                            WHERE to_date(dt) >= '{}' AND to_date(dt) <= '{}'
                        )
    {}
    AND to_date(citationusage.event_time) >= '{}'
    AND to_date(citationusage.event_time) <= '{}'
    AND useragent_is_bot = FALSE
    AND session_id in (
        SELECT session_id
        FROM citationusagepageload
        WHERE wiki = 'enwiki'
        {}
        AND to_date(event_time) >= '{}'
        AND to_date(event_time) <= '{}'
        AND useragent_is_bot = FALSE
        )
GROUP BY wpm_sections.section_h2, action
ORDER BY count desc
"""

pm_section_events = spark.sql(
    pm_section_events_query.format(
        start_date_string, end_date_string,
        event_exclusion_sql, start_date_string, end_date_string,
        pageload_exclusion_sql, start_date_string, end_date_string,
    ))
pm_section_events_rdd = pm_section_events.rdd
pm_section_events_df = sqlContext.createDataFrame(pm_section_events_rdd)
pm_section_events_pandas = pm_section_events_df.toPandas()


In [5]:
section_pda = pm_section_events_pandas.copy()
# replace 'NaN' section_h2 with 'None'
section_pda.section_h2.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered = section_pda.query('count>1000').copy()
# set precision before pivot
df_filtered['count'] = df_filtered['count'].map(lambda x: '{0:.0f}'.format(x))
display(Markdown("**Table I**: Total count of events (by all event types) for each top-level (H2) section ID for WP:M pages only. Limited to >= 1000 events."))
display(Markdown('_Numbers may not exactly match "raw" data (table II) because some section IDs could not be mapped to H2 section IDs (changed section ID, missing WPM page, etc.)_'))
display(Markdown('***missing*** values are largely because section IDs were not recorded "if the section is the Main Section" as per Schema:CitationUsage.'))
df_filtered.pivot(index='section_h2', columns='action', values='count')

**Table I**: Total count of events (by all event types) for each top-level (H2) section ID for WP:M pages only. Limited to >= 1000 events.

_Numbers may not exactly match "raw" data (table II) because some section IDs could not be mapped to H2 section IDs (changed section ID, missing WPM page, etc.)_

***missing*** values are largely because section IDs were not recorded "if the section is the Main Section" as per Schema:CitationUsage.

action,extClick,fnClick,fnHover,upClick
section_h2,,,,
-- missing --,37057,93880,133710,None
Adverse_effects,None,2109,3785,None
Background,None,None,1211,None
Cause,None,2374,5726,None
Causes,None,5255,10052,None
Chemistry,None,None,1100,None
Classification,None,None,2108,None
Diagnosis,None,4042,9029,None
Effects,None,None,1181,None


In [6]:
# "raw" section data from captured events to show extent of the capture issue described above
pm_section_events_raw_query = """
SELECT section_id, action, count(*) count
FROM 
    citationusage 
WHERE page_id IN (
                        SELECT DISTINCT page_id 
                        FROM ryanmax.projmed_with_extlinks
                        WHERE to_date(dt) >= '{}' AND to_date(dt) <= '{}'
                        )
    AND wiki = 'enwiki'
    {}
    AND to_date(event_time) >= '{}'
    AND to_date(event_time) <= '{}'
    AND useragent_is_bot = FALSE
    AND session_id in (
        SELECT session_id
        FROM citationusagepageload
        WHERE wiki = 'enwiki'
        {}
        AND to_date(event_time) >= '{}'
        AND to_date(event_time) <= '{}'
        AND useragent_is_bot = FALSE
        )
GROUP BY section_id, action
ORDER BY count desc
LIMIT 100
"""

pm_section_events_raw = spark.sql(
    pm_section_events_raw_query.format(
        start_date_string, end_date_string,
        event_exclusion_sql, start_date_string, end_date_string,
        pageload_exclusion_sql, start_date_string, end_date_string,
    ))
pm_section_events_raw_rdd = pm_section_events_raw.rdd
pm_section_events_raw_df = sqlContext.createDataFrame(pm_section_events_raw_rdd)
pm_section_events_raw_pandas = pm_section_events_raw_df.toPandas()
#pm_section_events_raw_pandas
#pm_section_events_pandas.pivot(index='section_id', columns='action', values='count')

In [7]:
section_pda_raw = pm_section_events_raw_pandas.copy()
# replace 'NaN' section_id with 'None'
section_pda_raw.section_id.fillna(value='-- missing --', inplace=True)
# limit to counts of 1K or more
df_filtered_raw = section_pda_raw.query('count>1000').copy()
# set precision before pivot
df_filtered_raw['count'] = df_filtered_raw['count'].map(lambda x: '{0:.0f}'.format(x))
display(Markdown("**Table II**: Raw total count of events (by all event types) for each section ID for WP:M pages only. Limited to >= 1000 events."))
df_filtered_raw.pivot(index='section_id', columns='action', values='count')

**Table II**: Raw total count of events (by all event types) for each section ID for WP:M pages only. Limited to >= 1000 events.

action,extClick,fnClick,fnHover,upClick
section_id,,,,
-- missing --,36996,93425,133043,None
Adverse_effects,None,1157,1679,None
Cause,None,None,2076,None
Causes,None,2875,4731,None
Classification,None,None,1711,None
Diagnosis,None,1870,3742,None
Epidemiology,None,2084,4027,None
External_links,29687,None,None,None
Further_reading,3302,None,None,None
